In [20]:
using Plots
#using Roots
#using BenchmarkTools
using Statistics

## 3.1 Armado de la red y cálculo de la energía del sistema
Consideren una red cuadrada de $N\times  N$, con $N^2$ el número de átomos que componen al sistema. 

In [2]:
#También declaramos las constantes
#definimos la constante de boltzman como 1 para que los resultados tengan mejor visualización
kb = 1
#Se declara con esee valor J por consenso del grupo :)
J=0.5;

In [3]:
function Malla_fria(N)
    return  [1 for i in 1:N^2]
end

Malla_fria (generic function with 1 method)

In [4]:
function Malla(N)
    return  [rand([-1,1]) for i in 1:N^2]
end

Malla (generic function with 1 method)

Generen una lista con los primeros vecinos de cada sitio implementando condiciones periódicas a la frontera (lo cual es equivalente a transformar al plano en un toro). De esta forma, todos los sitios de la red tienen 4 vecinos.

In [5]:
function Vecinos(N) #Vecinos
    N² = N*N
    v = zeros(Int,(N²,4))
    #Neighbor.site: (up, right,  down, left)
    for j in 1:N²
    #up
        if j+N<= N²
            v[j,1] = j+N 

        elseif j+N>N²
            v[j,1] = j - N*(N-1)
        end
    #right
        if j%N != 0 
            v[j,2] = j+1
        elseif j%N ==0
            v[j,2] = j-N+1 
        end
    #down
        if j-N>=1
            v[j,3] = j-N
        elseif j-N<1
            v[j,3] =j+N*(N-1)
        end
    #left    
        if (j-1)%N != 0
            v[j,4] = j-1
        elseif (j-1)%N ==0
            v[j,4] = j+N-1
        end
                
    end
            
    return v
end

Vecinos (generic function with 1 method)

In [23]:
Red = Malla(3)
Vecinos(3)

9×4 Matrix{Int64}:
 4  2  7  3
 5  3  8  1
 6  1  9  2
 7  5  1  6
 8  6  2  4
 9  4  3  5
 1  8  4  9
 2  9  5  7
 3  7  6  8

Ya que hayan hecho esto, calculen la energía de un par de conguraciones de espines y un valor de J.

La energía del sistema si hacemos la aproximación a los primero 4 vecinos es:

$$E=\sum _{<i,j>}-Js_{i}s_{j}$$

In [7]:
function Energia_total(red_lista,J)
    N² = length(red_lista)
    #generamos los indices de los vecinos
    vecinos_lista = Vecinos(Int(sqrt(N²)))
    #Creamos un arreglo donde iremos sumando cada elemento de la suma de la energía total
    energia_total = 0
    #Creamos un arreglo que obtenga la energía de interacción de cada elemento de la red
    for i in 1:N²
        #Obtenemos los indices de los vecinos
        vecinosi= vecinos_lista[i]
        for j in vecinosi
            energia_total += red_lista[i]*red_lista[j]
        end
    
    end
    return energia_total*(-J)
end

Energia_total (generic function with 1 method)

In [8]:
function magnetizacion(red_lista)
    return sum(red_lista)
end

magnetizacion (generic function with 1 method)

In [9]:
for i in 1:4
    print(i)
end

1234

In [25]:
espines =  Malla(3)
J=0.5
vecinos_lista = Vecinos(3)
for j in 1:9
    for i in 1:4 #Multiplicamos spin de la malla por sus vecinos
        dE = 2*J*espines[j]*espines[Int(vecinos_lista[j][i])]
    end
end
dE

LoadError: BoundsError

In [11]:
rand(1)

1-element Vector{Float64}:
 0.3016701885416173

In [12]:
function Termalizacion(kt, espines, Q, J)
    N = Int(sqrt(length(espines)))
    N² = N*N
    Flip = []
    flip_contador = 0
    vecinos_lista = Vecinos(N)
    efinc = []
    magnefinc = []
    dE = 0
        for j in 1:N²
        #Obtenemos los indices de los vecinos
        vecinosj= vecinos_lista[j]
        for i in vecinosj
            dE += 2*J*espines[j]*espines[i]
            pob = exp(-dE/kt)
            if dE<0 || pob>rand(1)
                espines[j] = -espines[j]
                flip_contador += 1
            end
                en = energia(espines, 0.5)
                m = sum(espines)
                energias = append!(energias, en)
                magne = append!(magne, m)
                Flip = append!(Flip,flip_contador)
        end
        enfinc = append!(enfinc, energias)
        magnefinc = append!(magnefinc, abs(magne))
        end
    return Flip, enfinc, magnefinc
end

Termalizacion (generic function with 1 method)

In [13]:
Red20 = Malla_fria(20) 
Red40 = Malla_fria(40) 
Red50 = Malla_fria(50) 
Red100 = Malla_fria(100);

In [14]:
Flip20, E20, M20 = Termalizacion(1.5, Red20, 7000, 0.5)
Flip40, E40, M40 = Termalizacion(1.5, Red40, 7000, 0.5)
Flip50, E50, M50 = Termalizacion(1.5, Red50, 7000, 0.5)
Flip100, E100, M100 = Termalizacion(1.5, Red100, 7000, 0.5);

LoadError: MethodError: no method matching isless(::Vector{Float64}, ::Float64)
[0mClosest candidates are:
[0m  isless([91m::T[39m, ::T) where T<:Union{Float16, Float32, Float64} at ~/julia-1.7.2-linux-x86_64/julia-1.7.2/share/julia/base/float.jl:460
[0m  isless([91m::Union{StatsBase.PValue, StatsBase.TestStat}[39m, ::AbstractFloat) at ~/.julia/packages/StatsBase/Ya5vt/src/statmodels.jl:99
[0m  isless([91m::Union{StatsBase.PValue, StatsBase.TestStat}[39m, ::Real) at ~/.julia/packages/StatsBase/Ya5vt/src/statmodels.jl:90
[0m  ...

In [15]:
plot!(Flip20, E20./20^2,label = "Malla de 20x20")
plot!(Flip40, E40./40^2,label = "Malla de 40x40")
plot!(Flip50, E50./50^2,label = "Malla de 50x50")
plot!(Flip100, E100./100^2,label = "Malla de 100x100")
plot!(xlabel = "Spin flips", ylabel ="<ϵ>")

LoadError: UndefVarError: E20 not defined

In [16]:
Red20_f = Malla_fria(20)
Red40_f = Malla_fria(40)
Red50_f = Malla_fria(50)
Red100_f = Malla_fria(100);

In [17]:
Flip20_f, E20_f, M20_f = Termalizacion(1.5, Red20_f, 7000, 0.5)
Flip40_f, E40_f, M40_f = Termalizacion(1.5, Red40_f, 7000, 0.5)
Flip50_f, E50_f, M50_f = Termalizacion(1.5, Red50_f, 7000, 0.5)
Flip100_f, E100_f, M100_f = Termalizacion(1.5, Red0100, 7000, 0.5);

LoadError: MethodError: no method matching isless(::Vector{Float64}, ::Float64)
[0mClosest candidates are:
[0m  isless([91m::T[39m, ::T) where T<:Union{Float16, Float32, Float64} at ~/julia-1.7.2-linux-x86_64/julia-1.7.2/share/julia/base/float.jl:460
[0m  isless([91m::Union{StatsBase.PValue, StatsBase.TestStat}[39m, ::AbstractFloat) at ~/.julia/packages/StatsBase/Ya5vt/src/statmodels.jl:99
[0m  isless([91m::Union{StatsBase.PValue, StatsBase.TestStat}[39m, ::Real) at ~/.julia/packages/StatsBase/Ya5vt/src/statmodels.jl:90
[0m  ...

In [18]:
plot!(Flip20_f,E20_f./(20^2), label = "Malla de 20x20")
plot!(Flip40_f,E40_f./(40^2), label = "Malla de 40x40")
plot!(Flip50_f,E50_f./(50^2), label = "Malla de 50x50")
plot!(Flip100_f,E100_f./(100^2), label = "Malla de 100x100")
plot!(title = "Configuración caliente", xlabel = "Spin flips", ylabel ="<ϵ>")

LoadError: UndefVarError: E20_f not defined